# COMP 790.139: Natural Language Processing (Fall 2017): Coding Homework 2 (Sequence-to-Label Learning, Entailment Recognition)
Created by TA Yixin Nie (Instructor: Mohit Bansal)


# Instructions

**All the instructions are present in the jupyter notebook (as shown in the class; and see the preview below).  
Install jupyter notebook in your python environment and download the file below.**  
https://drive.google.com/drive/folders/0B6i0pVGwapCdVHV1cnVNdWhvM1k?usp=sharing  
  
  
**Use this directory as your workspace and write your code in the “hw2.ipynb” file. You could also add extra images or tables in the directory and link them into “hw2.ipynb” file but grading will only base on the “hw2.ipynb” file.**    

**Make sure to name your directory as `"<your_name>_hw2"` and compress it to `"<your_name>_hw2.zip"`.   
Email the file to <a href="mailto:comp790.hw@gmail.com">comp790.hw@gmail.com</a> for submission.**    

------

# Homework 2 Preview
The main goals for homework 2 are:
1. Going through some setup procedures of training deep neural network for NLP.
2. Getting familiar with Recurrent Neural Networks.
3. Learning to handle variable length inputs in deep learning framework (Pytorch, Tensorflow).
4. Building a sequence-to-label model for natural language inference task.
5. Learning to improve your model by interpreting your experiment result.

Notice:  
You can test or run your code in any environment but you could only show your codes, your results and your write-ups in this single notebook file. We will not re-run your code for grading.  

<span style="color:red;font-size:16px">**Fill in the #TODOs and try to stick to the provided APIs.**</span>

In [1]:
import json
import numpy as np
from tqdm import tqdm
import gc
import re
import spacy
import torch
import time

## 1. Loading and cleaning raw dataset (3 pt)
In this section, you are required to obtain the provide vocabulary and load and clear the raw SNLI dataset.  

The original SNLI dataset contains about 50k/10k/10k train/dev/test sentences pairs. We also provide smaller training set that contains 0.05 of the original training set in `'snli_1.0/small_0.05_snli_train.jsonl'`. You could pick one training set depending on your computation resources. Each line of the dataset file is a data point in JSON format. The `'gold_label'` field contains the label (`'entailment', 'neutral' or 'contradiction'`) for this data point. There are some data points that don't have the gold_label (their value for the gold label field is `'-'`). We will only use those examples which have a gold label. For a detailed description of the task and the dataset file, please refer to https://nlp.stanford.edu/projects/snli/.

The vocabulary we will use for this homework is in the `vocabu.txt` file. It contains 19,007 tokens and all the missing words in the snli data set should be mapped to the last token in the `vocabu.txt` file which is `'<unk-0>'`.

Finish the preprocessing code in the `obtain_data()` function.

In [2]:
def obtain_data(file_name='snli_1.0/small_0.05_snli_train.jsonl', vocab_file_name='vocabu.txt'):
    json_data = []
    
    with open(file_name) as data_file:
        for l in data_file:
            json_data.append(json.loads(l))
    
    data = []
    for item in json_data:
        if item['gold_label'] != '-':
            example = (item['sentence1'], item['sentence2'], item['gold_label'])
            data.append(example)
    
    stoi = []
    with open(vocab_file_name) as voc_file:
        for token in voc_file:
            stoi.append(token[:-1]) # remove escape characters
    
    return data, stoi

Run the code below (according to your choice of training set) and show how many data points are loaded.

In [3]:
data, stoi = obtain_data('snli_1.0/small_0.05_snli_train.jsonl')
print('# of words in vocabulary')
print(len(stoi))
print('snli_1.0/small_0.05_snli_train.jsonl')
print(len(data))

#print('snli_1.0/snli_1.0_train.jsonl')
#data, stoi = obtain_data('snli_1.0/snli_1.0_train.jsonl')
#print(len(data))

# of words in vocabulary
19007
snli_1.0/small_0.05_snli_train.jsonl
27284


## 2. Build your vocabulary (6 pt)
In this section, we will build a vocabulary python object and load pretrained Glove Embedding for your vocabulary.   Both of them will be important throughout this assignment.  

The `Vocabulary` object will have three attributes, namely `embeddings`, `itos` and `stoi`.  

* `stoi` is a python list which can be used to map a word into a unique id.  
* `itos` is a python dictionary which can be used to map a given id into the corresponding word.  
* `embeddings` is a numpy array with shape: (size_of_vocabulary, embedding_dimension). Note: The `i`th row of your embeddings will be the vector for the word whose id is `i`.

Example:
```
embeddings[stoi['happy']] # This will give you the corresponding vector for word 'happy'.
itos[100] # This will give you the word whose id is 100.
```

In the cell below:
1. Directly use `stoi` to (you loaded in the last section) initiate your `vocabulary.stoi` attribute. (0.5 pt)
2. Build `itos` according to the `stoi`. (0.5 pt)
3. Load Glove embedding in the function `set_word_embedding(self, embedding_file)` and initiate your embeddings using pretrain Glove embedding. For detailed description and format of Glove Embeddings, refer to https://nlp.stanford.edu/projects/glove/. (4 pt)
4. Numericalize your data in the `process_data(self, data)` function, namely convert the sentences into a list of ids and the labels into label_ids. (Use the dictionary `label_id` in the cell below for mapping.) (1 pt)

Remember that your output of `process_data(self, data)` should be a list of triple `[(token_id_list, token_id_list, int)]`.

In [4]:
label_id = {'entailment': 0, 'neutral': 1, 'contradiction': 2}

class Vocabulary(object):
    def __init__(self, stoi):
        self.embeddings = None
        self.itos = dict(zip(stoi,range(len(stoi))))
        self.stoi = stoi
    
    def set_word_embedding(self, embedding_file):
        
        f = open('glove.840B.300d.txt','r', encoding="utf8")
        model = {}
        count = 0
        for line in f:
            splitLine = re.split(', | |\n',line)
            word = splitLine[0]
            embedding = np.array([np.float32(val) for val in splitLine[1:-1]])
            model[word] = embedding
        
        self.embeddings = np.array([])
        for item in self.stoi:
            if item in model.keys():
                self.embeddings = np.append(self.embeddings, [[model[item]]])
            else:
                self.embeddings = np.append(self.embeddings, \
                                [[np.array(np.random.uniform(-1,1,300), dtype = np.float32)]])

    '''
    def dummy_embedding(self, embed_d=300):
        # This is only a dummy embedding using random initialization. Do not use this function in the assignment.
        for i, w in enumerate(self.stoi):
            self.itos[w] = i
        vocab_size = len(self.stoi)
        self.embeddings = np.asarray(np.random.randn(vocab_size, embed_d), dtype=np.float32)
    '''    
    def process_data(self, data):
        numerialized_data = []
        nlp = spacy.load('en')
        tokenization_1 = list()
        tokenization_2 = list()
        for s1, s2, y in data:
            doc_1 = nlp(s1)
            doc_2 = nlp(s2)
            tmp_list_1, tmp_list_2, n_s1, n_s2 = list(),list(),list(),list()
            
            for token in doc_1:
                item = token.orth_
                if item in self.itos.keys():
                    n_s1.append(self.itos[item])
                else:
                    n_s1.append(self.itos[self.stoi[-1]])
                    
            for token in doc_2:
                item = token.orth_
                if item in self.itos.keys():
                    n_s2.append(self.itos[item])
                else:
                    n_s2.append(self.itos[self.stoi[-1]])
            
            n_y = label_id[y]
            
            numerialized_data.append((n_s1, n_s2, n_y))
            
        return numerialized_data

In [5]:
# You can use this code to check the correctness of your code.

vocab = Vocabulary(stoi=stoi)
vocab.set_word_embedding('glove.840B.300d.txt')
n_data = vocab.process_data(data)

print(vocab.embeddings.shape)
print(len(n_data))

print(vocab.itos['hello'])
print(vocab.stoi[6030])

(5702100,)
27284
6030
hello


In [6]:
vocab.embeddings = vocab.embeddings.reshape((19007,300))
print(vocab.embeddings.shape)

(19007, 300)


## 3. Padding (5 pt)
For a neural network model to be trained on sequential natural language data, we will need to pad each sequence into a fixed length.

The problem with sequential data for neural networks is that different examples might have different lengths. So we will need to pad all the data (or the data in a batch) to a fixed length (say 50) for parallel computation.

In the cell below, finish the `convert_to_numpy(data, padding_length=50)` function which can convert the data into numpy ndarray and with padding. We recommend padding value to be zero.  

You can do padding in corpus level (that is padding all the example in the dataset to a fixed length which means that all the example in your dataset will have the same length) or batch level (padding the example in a batch to a fixed length, different batch might have different length)

Some Neural Network framework (like tensorflow) use a more advanced batching technique call bucketing. We will not implement bucketing in this assignment but refer to https://www.tensorflow.org/tutorials/seq2seq for more details.

The padded data could be something like this:
```
for start, end in batch_index_gen(3, len(y)):
    print(s1[start:end])
    print(s2[start:end])
    print(y[start:end])
    break
```
Output:
```
[[   5   26    7   99   10  281    6    8  293   13    4   39  388    3    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0]
 [  61  147    9    7  282  104 1797   19    4  242    3    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0]
 [   5    9   11    4   16  586    8   45    6   46   13    4 1043   11 8421  242    3    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0]]
[[  15   26    7   22  593  480    3    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0]
 [   5    9    6    4  242    7  164   40   23  682   18 2635    3    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0]
 [5734    4  130   13 5079   17   52   20  152  234    8   45    3    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0]]
[1 1 0]
```

You don't have to have the same output. All you will need to do is to convert your data into the format such that it can be feeded into a neural network.

In [7]:
def batch_index_gen(batch_size, size):
    batch_indexer = []
    start = 0
    while start < size:
        end = start + batch_size
        if end > size:
            end = size
        batch_indexer.append((start, end))
        start = end
    return batch_indexer

def convert_to_numpy(data, padding_length=50):
    total_size = len(data)
    s1_ = []; s2_ = []; y_ = []
    for i, (s1, s2, y) in enumerate(data):
        if len(s1) <= padding_length:
            s1 =  s1 + [0] *(50-len(s1))
        else:
            s1 = s1[:padding_length]
        
        if len(s2) <= padding_length:
            s2 =  s2 + [0] *(50-len(s2))
        else:
            s2 = s2[:padding_length]
        
        s1_.append(s1)
        s2_.append(s2)
        y_.append(y)
    return s1_, s2_, y_

In [8]:
s1, s2, y = convert_to_numpy(n_data, 50)

In [9]:
for start, end in batch_index_gen(3, len(y)):
    print(s1[start:end])
    print(s2[start:end])
    print(y[start:end])
    break

[[5, 26, 7, 99, 10, 281, 6, 8, 293, 13, 4, 39, 388, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [61, 147, 9, 7, 282, 104, 1797, 19, 4, 242, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [5, 9, 11, 4, 16, 586, 8, 45, 6, 46, 13, 4, 1043, 11, 8421, 242, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[15, 26, 7, 22, 593, 480, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [5, 9, 6, 4, 242, 7, 164, 40, 23, 682, 18, 2635, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [5734, 4, 130, 13, 5079, 17, 52, 20, 152, 234, 8, 45, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[1, 1, 0]


## 4. RNN Sequence-to-label Model (15 pt + 5 bonus)
It's time to build your RNN sequence-to-label model.

Remember that the input of the model is two sequences and you will use the label to train your model.  
**RNN component is mandatory** for the model in this assignment.

Recommended reading: 
* http://colah.github.io/posts/2015-08-Understanding-LSTMs/
* https://discuss.pytorch.org/t/batch-processing-with-variable-length-sequences/3150 (variable length handling in pytorch)
* https://danijar.com/variable-sequence-lengths-in-tensorflow/ (variable length handling in tensorflow)

Bonus will be given to those who correctly handle variable length inputs.

In [10]:
        # inputs of lstm : input, (h_0, c_0)
        #   input: (seq_len, batch, input_size)
        #   h_0 : number_layers* num_directions, batch, hidden_size 
        #         here we have one layer and one direction. 
        
        #  c_0 : numb_layers*num_directions, batch, hidden_size
        
        # output of lstm :
        #   h_n and c_n : same to the h_0 and c_0
        #   lstm_out : (seq_len, batch, hidden_size*num_dir)
        
                # 根据lstm的输入要求， 第一个其实是h0的size 第二个是c0的size
        #>>> rnn = nn.LSTM(10, 20, 2)   # 2 代表了要么是两层！ 
                            # 两个10分别代表 input_size, hidden_size
        #>>> input = Variable(torch.randn(5, 3, 10)) # 5,3,10 代表了 seq_len = 5, 
                                                     # batch_size = 3, input_size(句长)= 10
        #>>> h0 = Variable(torch.randn(2, 3, 20))
        #>>> c0 = Variable(torch.randn(2, 3, 20))
        #>>> output, hn = rnn(input, (h0, c0))

In [11]:
import torch
import torch.nn as nn
import torch_util
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

class LSTMModel(nn.Module):
    # TODO
    #   Build your model in this cell.
    
    def __init__(self, h_size=128, d=300, label_size = 3, v_size = 10, batch_size = 20):
        super(LSTMModel, self).__init__()
        self.hidden_size = h_size
        self.embedding_dim = d
        self.label_size = label_size
        self.v_size = v_size
        self.batch_size = batch_size
        
        self.embedding = nn.Embedding(v_size, d)
        self.lstm = nn.LSTM(input_size= d, hidden_size= h_size, num_layers= 1)
        self.hidden2tag = nn.Linear(h_size*100, h_size*100)
        self.linear2 = nn.Linear(h_size*100, label_size)
        
    def display(self):
        for param in self.parameters():
            #print(param)
            print(param.data.size())
            
    def init_hidden(self):
        return (Variable(torch.zeros(1, self.batch_size, self.hidden_size)).cuda(),#那个1 代表了layer×dim,
                Variable(torch.zeros(1, self.batch_size, self.hidden_size)).cuda())#.cuda())

    def forward(self, sentence1_var, sentence2_var, hidden1, hidden2):
        embeds1 = self.embedding(sentence1_var).type_as(torch.FloatTensor()).cuda()#.cuda()
        embeds2 = self.embedding(sentence2_var).type_as(torch.FloatTensor()).cuda()#.cuda()
        
        lstm1_out, hidden1 = self.lstm(embeds1.view(len(sentence1_var[0]),self.batch_size,self.embedding_dim), hidden1)
        lstm2_out, hidden2 = self.lstm(embeds2.view(len(sentence2_var[0]),self.batch_size,self.embedding_dim), hidden2)
        
        
        combined  = torch.cat((lstm1_out.view(20,-1), lstm2_out.view(20,-1)),1).cuda()
        tag_space = F.sigmoid(self.hidden2tag(combined)).cuda()
        tag_space2 = F.sigmoid(self.linear2(tag_space)).cuda()#.cuda()
        
        probs = F.softmax(tag_space).cuda()#.cuda()
        

        return probs, hidden1, hidden2

In [12]:
from random import shuffle

In [14]:
batch_size = 20
model = LSTMModel(batch_size= batch_size)
model.embedding.weight.data = torch.from_numpy(vocab.embeddings)
model.cuda()#.cuda()

optimizer = optim.Adam(model.parameters(), lr=0.03)
loss_function = nn.CrossEntropyLoss() 
loss_function.cuda()#.cuda()

def repackage_hidden(h):
    """Wraps hidden states in new Variables, to detach them from their history."""
    if type(h) == Variable:
        return Variable(h.data)
    else:
        return tuple(repackage_hidden(v) for v in h)


total_loss = 0
hidden1 = model.init_hidden()
hidden2 = model.init_hidden()

for epoch in range(20):
    s1_shuffle = []
    s2_shuffle = []
    y_shuffle = []
    index_shuf = list(range(len(s1)))
    shuffle(index_shuf)
    for i in index_shuf:
        s1_shuffle.append(s1[i])
        s2_shuffle.append(s2[i])
        y_shuffle.append(y[i])
    
    
    for i in range(len(y)//batch_size):
        s1_ = Variable(torch.LongTensor(s1_shuffle[i*batch_size: (i+1)*batch_size])).cuda()
        s2_ = Variable(torch.LongTensor(s2_shuffle[i*batch_size: (i+1)*batch_size])).cuda()
        y_  = Variable(torch.LongTensor(y_shuffle[i*batch_size: (i+1)*batch_size])).cuda()
    
        hidden1 = repackage_hidden(hidden1)
        hidden2 = repackage_hidden(hidden2)
        
        model.zero_grad()
    
        probs, hidden1, hidden2 = model(s1_, s2_, hidden1, hidden2)
        #probs = model(s1_, s2_)
        
        loss = loss_function(probs, y_)
        
        loss.backward()
        #loss.backward()  # 这个地方之前为啥要retain 是因为每次的hidden都是一个zero tensor 没办法backward（）
        optimizer.step()
        
        total_loss += loss.data
        if i%100 == 0:
            print('Time used is %d' %(time.time() - start))
            print('Total loss is %s' %total_loss)
            total_loss = 0
            start = time.time()

Time used is 1511144257
Total loss is 
 9.4572
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Time used is 13
Total loss is 
 945.7078
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Time used is 13
Total loss is 
 945.7076
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Time used is 13
Total loss is 
 945.7076
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Time used is 13
Total loss is 
 945.7076
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Time used is 13
Total loss is 
 945.7076
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Time used is 13
Total loss is 
 945.7076
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Time used is 13
Total loss is 
 945.7076
[torch.cuda.FloatTensor of size 1 (GPU 0)]



KeyboardInterrupt: 

In [215]:
use_cuda = False
def repackage_hidden(h):
    if type(h) == Variable:
        if use_cuda:
            return Variable(h.data).cuda()
        return Variable(h.data)
    else:
        return tuple(repackage_hidden(v) for v in h)

embedding_layer = nn.Embedding(19007, 300)
batch_size = 20
input = Variable(torch.LongTensor(s1[0:batch_size])) #原来是2*50*300 用了view改成 50*2*300
intermediate = embedding_layer(input).view(len(s1[0]),batch_size,300)
hidden = (Variable(torch.zeros(1, batch_size, 128)), Variable(torch.zeros(1, batch_size, 128)))
lstm = nn.LSTM(300,128,1)
lstm_out, hidden_ = lstm(intermediate, hidden)
#print(lstm_out.view(-1,)

intermediate2 = torch.cat((lstm_out.view(20,-1), lstm_out.view(20,-1)),1)# 2*256
#print(intermediate2)
hidden2tag = nn.Linear(128*100, 3)
a = F.sigmoid(hidden2tag(intermediate2))
print(a)
#hidden_ =  repackage_hidden(hidden_)

#lstm_out, hidden_ = lstm(intermediate, hidden_)
'''
model = LSTMModel()
model.embedding.weight.data = torch.from_numpy(vocab.embeddings)
model.display()
print(model)'''

Variable containing:
 0.4698  0.4955  0.5253
 0.5064  0.4754  0.4631
 0.4779  0.4982  0.4813
 0.4841  0.5005  0.4870
 0.4881  0.4904  0.4702
 0.5083  0.4725  0.4922
 0.5060  0.4988  0.4846
 0.4783  0.4957  0.4621
 0.5112  0.4621  0.4710
 0.5074  0.4944  0.4734
 0.5128  0.4784  0.4625
 0.4702  0.4998  0.4809
 0.5100  0.5079  0.4870
 0.5339  0.5171  0.5323
 0.5018  0.4885  0.4747
 0.4720  0.5224  0.4772
 0.4813  0.4844  0.4600
 0.4767  0.4835  0.4610
 0.4843  0.4899  0.4492
 0.5024  0.4847  0.4581
[torch.FloatTensor of size 20x3]



'\nmodel = LSTMModel()\nmodel.embedding.weight.data = torch.from_numpy(vocab.embeddings)\nmodel.display()\nprint(model)'

## 5. Training your model (3 pt)
You could now train your model batch by batch using whatever optimizer you want.  
In order to keep track of your training, you should also print out the loss every 1000*`X` batch.  

Write your code in the cell below. Print out the loss every 1000*`X` batch and your final average loss.

## 6. Evaluation (8 pt)

Complete the `eval_model(model, mode='dev')` function in the cell below for evaluate your model on dev and test set. The return value of this function should be the accuracy. Try to tune your model on the dev set and finally evaluate the model with best-dev-result on the test set and report the final test set result. **Note: You should try your model on test set only once.**

If you are not satisfied with the result from the evaluation (in most cases), you could try to make some changes in your model and re-try.

If you are running most things correctly, your result should easily be at least 60% on the dev set if you are using 0.05 of the SNLI training data;  and at least 80% on the dev set if you are using full training set.

In [80]:
def eval_model(model, mode='dev'):
    file_name = 'snli_1.0/snli_1.0_dev.jsonl' if mode == 'dev' else 'snli_1.0/snli_1.0_test.jsonl'
        
    dev_data, _ = obtain_data(file_name)
    dev_n_data = vocab.process_data(data)
    
    #`<your dev data>` = convert_to_numpy(dev_n_data)
    
    model.eval()
    new_dev_data = convert_to_numpy(vocab.process_data(dev_data), 50)
    s1 = []
    s2 = []
    y = []
    for item in new_data:
        s1.append(item[0])
        s2.append(item[1])
        y.append(item[2])

    y = [item for item in y]
    
    pred = np.argmax(model(autograd.Variable(s1), autograd.Variable(s2)).data.numpy(),axis = 1)

    
    total = 0
    hit = 0
    
    # TODO:
    #   write your code here to show the result of your model on 
    for i in range(len(label)):
        if label[i] == pred[i]:
            hit = hit + 1
        total = total + 1
    
    return hit / float(total)

In [5]:
# I can't train the model well. Will figure out in the weekend.

## 7. Additional analysis (5 pt)
This course is designed to train you as an NLP researcher. A researcher should not only be able to implement newly emerged models and algorithms and get them to work but also give reasons and intuitions behind every decision you make during your research (e.g. parameter and structure design).  
In this section, write down anything you think that is important in this homework.  
It could be:
* The problems you encountered during the implementation and how you resolve it.
* You were not satisfied with the results and you made some changes to (or it fails to) improve it. Why do you think those changes can (might) be helpful?  

Use your imagination and try to record every detail of your experiments. The bonus will be given to novel and reasonable thoughts.

In [ ]:
'''
1. I moved from win to linux for Pytorch. One thing is the GPU memory control for pytorch.
   I am not sure why, but I am asked to add 'retain_graph = True' in loss.backward, which incrase GPU memory 
   a lot and does not clear GPU memory when it is not used.
   
   So after 500 batch, OOM occured. I moved back to CPU.
   
2. Not very sure how to handle variable length. Or not sure why we need to pack the sentence and unpack them.
    Or the advantage to do that. So I used the general method.
    
3. Training speed. The training speed on my desktop is super slow. I don't understand why, but will keep working on 
    it this weekend.
    
4. I think we should do Bi directional LSTM. It is obvious some words after the target also provides information. 
    Using bi directional LSTM may help.

5. One thing I faced is I want to append <unk-0> when the model faced some words not in glove dictionary. Hoever, 
    that will introduce word with index 19008 which is out of range. I used long time to figure out the problem of
    this error.

6. Most of my time in these two weeks was spent on interview (I will graduate after this semester). I don't have 
    enough time to modify my model and code.
'''